In [1]:
from wikiparser_utils import WikiXMLDump, WikiPage
import os
import nltk
import numpy as np
from difflib import Differ 
from utils.difflibparser import DifflibParser, DiffCode

In [2]:
dump = WikiXMLDump('data/history.xml')

In [3]:
#dump.get_save_on_disk('data/pages')

In [4]:
from tqdm import tqdm

counter1 = 0
counter3 = 0
counter5 = 0
counter10 = 0
dirs = sorted(os.listdir('data/pages'))
for dir_ in tqdm(dirs):
    page_revisions = WikiPage().load_revisions(f"data/pages/{dir_}")
    if len(page_revisions) > 1:
        counter1 += 1
    if len(page_revisions) > 3:
        counter3 += 1
    if len(page_revisions) > 5:
        counter5 += 1
    if len(page_revisions) > 10:
        if not counter10:
            pass
            #print(dir_)
        counter10 += 1
    if True and len(page_revisions) > 30 and len(page_revisions) < 300:
        rev_list = sorted(list(page_revisions))
        if len(page_revisions[rev_list[-1]].get_links()[0]) > 10:
            print(dir_)
        

 13%|█▎        | 121/925 [00:10<00:53, 14.90it/s]

Anthony Laffranchi_2934955


 19%|█▉        | 180/925 [00:11<00:17, 43.70it/s]

Ben Galea_2935226


 31%|███       | 286/925 [00:24<01:05,  9.72it/s]

...

 67%|██████▋   | 623/925 [01:03<00:20, 14.75it/s]

Openwave_2935786


 70%|███████   | 650/925 [01:06<00:21, 13.08it/s]

Philippe Albert_2935301


100%|██████████| 925/925 [01:44<00:00,  8.81it/s]


In [5]:
counter1, counter3, counter5, counter10

(837, 679, 606, 521)

In [6]:
page_revisions = WikiPage().load_revisions(f"data/pages/New Balance_2935379")
rev_list = list(page_revisions)

In [7]:
rev_list[-5:], len(rev_list)

([1109774145, 1111429991, 1111497925, 1111954890, 1112319310], 1509)

In [21]:
last_revision = page_revisions[rev_list[-50]]
prelast_revision = page_revisions[rev_list[-100]]
lr_links, _ = last_revision.get_links()
plr_links, _ = prelast_revision.get_links()

In [22]:
def links_diff(links_old, links_new):
    set_old = set(links_old)
    set_new = set(links_new)
    set_diff = set_new - set_old
    return list(set_diff)

In [23]:
links_diff(plr_links, lr_links)

[('Rockport is a Massachusetts shoe company again',
  'http://www.bizjournals.com/boston/news/2015/08/03/rockport-is-a-massachusetts-shoe-company-again.html'),
 ('The Trump-Era Corporate Boycott',
  'http://www.newyorker.com/magazine/2017/01/09/the-trump-era-corporate-boycott'),
 ('Berkshire Partners and New Balance to Acquire The Rockport Company',
  'https://www.businesswire.com/news/home/20150123005093/en/Berkshire-Partners-and-New-Balance-to-Acquire-The-Rockport-Company'),
 ("Fast times for New Balance: New Methuen plant to employ 200 by year's end on strong demand - The Boston Globe ",
  'https://www.bostonglobe.com/2022/03/28/business/fast-times-new-balance-new-methuen-plant-employ-200-by-years-end-strong-demand/'),
 ("2 LI shoe stores close in parent's bankruptcy}}</ref>",
  'https://www.newsday.com/business/rockport-tanger-roosevelt-field-1.20152428'),
 ('Jack Harlow Signs with New Balance ',
  'https://www.sneakerfreaker.com/news/jack-harlow-new-balance-sneaker-deal'),
 ('Shoe

In [24]:
def text_diff(old_rev, new_rev):
    old_sec, _ = old_rev.get_clean_sections()
    new_sec, _ = new_rev.get_clean_sections()
    set_old = set(old_sec)
    set_new = set(new_sec)
    set_diff = set_new - set_old
    return list(set_diff)

In [25]:
td = text_diff(prelast_revision, last_revision)
print(len(td), '\n\n')
print(td[0])

16 



(' Players ', '== Players ==\n*  Claudinho\n*  Bukayo Saka\n*   Eberechi Eze\n*  Harvey Elliott\n*  Raheem Sterling\n*  Rose Lavelle\n*  Sadio Mane\n*  Toni Martinez\n*  Yeferson Soteldo\n\n')


In [27]:
old_text = prelast_revision.get_plain_text()
new_text = last_revision.get_plain_text()
sent_text_old = nltk.sent_tokenize(old_text)
sent_text_new = nltk.sent_tokenize(new_text)

d = Differ()
result = list(d.compare(sent_text_old, sent_text_new))

if False:
    for line in result: 
        if '+' == line[0]:
            print(line)

    for line in result: 
        if '-' == line[0]:
            print(line)

In [28]:
def text_diff_diff(old_rev, new_rev, sentence_tokenizer=nltk.sent_tokenize, epsilon=2, beta=2*2):
    d = Differ()
    old_text = old_rev.get_plain_text()
    new_text = new_rev.get_plain_text()
    
    sent_text_old = sentence_tokenizer(old_text)
    sent_text_new = sentence_tokenizer(new_text)
    
    old_sents = []
    for i in range(epsilon, len(sent_text_old) - epsilon):
        old_sents.append(' '.join(sent_text_old[(i-epsilon):(i+epsilon+1)]))
    if not old_sents:
        old_sents = [' '.join(sent_text_old)]
        
    new_sents = []
    for i in range(epsilon, len(sent_text_new) - epsilon):
        new_sents.append(' '.join(sent_text_new[(i-epsilon):(i+epsilon+1)]))
    if not new_sents:
        new_sents = [' '.join(sent_text_new)]
        
    
    result = []
    dif_result = list(DifflibParser(old_sents, new_sents))
    old_text, new_text, last_diff_id = [], [], -1000
    for dif_id, dif_line in enumerate(dif_result):
        if dif_line['code'] != DiffCode.SIMILAR:
            if np.abs(dif_id - last_diff_id) > beta:
                result.append(dif_line)  
                last_diff_id = dif_id
    return result

In [42]:
def get_changes(diffs):
    all_changes = {}
    for diff_id, diff_obj in enumerate(diffs):
        if diff_obj['code'] == DiffCode.RIGHTONLY:
            all_changes[diff_id] = ([diff_obj['line']], 'r')
        elif diff_obj['code'] == DiffCode.LEFTONLY:
            all_changes[diff_id] = ([diff_obj['line']], 'l')
        elif diff_obj['code'] == DiffCode.CHANGED:
            r_change = diff_obj['rightchanges']
            cur_ch = -10
            prev_ch = -10
            all_r_changes = []
            for ch in r_change:
                if prev_ch < 0:
                    prev_ch = ch
                    cur_ch = ch
                if np.abs(ch - cur_ch) > 1:
                    new_change = diff_obj['newline'][prev_ch:cur_ch+1]
                    all_r_changes.append(new_change)
                    prev_ch = ch
                cur_ch = ch
            new_change = diff_obj['newline'][prev_ch:cur_ch+1]
            all_r_changes.append(new_change)
            all_changes[diff_id] = (all_r_changes, 'c')
    return all_changes

In [49]:
diffs = text_diff_diff(last_revision, prelast_revision)
print('Total num of diffs:', len(diffs), '\n')
for k, v in get_changes(diffs).items():
    v_str = '\n'.join(v[0])
    print(f"Key {k}_{v[1]}:\n{v_str}\n\n")

Total num of diffs: 14 

Key 0_c:
 that includes brands Aravon, Dunham, and New Balance


Key 1_c:
Billion Dollars in
0 and are made up of


Key 2_c:
 February
5
the company announced its entry into the global soccer (association football) mar
et


Key 3_l:
DeMartini's background includes Procter & Gamble and Gillette Shave Company. Today, 30 percent of the New Balance shoes sold in the European market are manufactured at the New Balance facility in Flimby, England. In 2011 New Balance placed its Aravon, Cobb Hill, and Dunham brands under its Drydock Footwear affiliate. In 2015, New Balance's parent, together with Berkshire Partners, bought the  Rockport shoe company from the Adidas Group and combined it with Drydock Footwear under the name The Rockport Group, now with the Aravon, Cobb Hill, Dunham, and Rockport brands. In 2018, The Rockport Group went into bankruptcy and was sold.


Key 4_c:
the company announced its entry into the global soccer (association football) market. 
had sta